In [81]:
# gaussian model
from sklearn.naive_bayes import GaussianNB as gnb
import pandas as pd
from sklearn.datasets import load_breast_cancer as lbc
from sklearn.model_selection import train_test_split as tts
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.linear_model import LogisticRegression as lr
from sklearn import metrics as m
from sklearn.preprocessing import MinMaxScaler as mms
import numpy as np
MMS = mms()
dataset = lbc(as_frame=True)
target = dataset.target.values
features = dataset.data.values
features = MMS.fit_transform(features)
f_train, f_test, t_train, t_test = tts(features, target, random_state=0)

gaussionModel = gnb()
# gaussionModel.fit(f_train, t_train)
# pred_t = gaussionModel.predict(f_test)
# print("Accuracy: ",m.accuracy_score(t_test,pred_t))
# print(gaussionModel.predict(f_test))

decissionTreeModel = dt()
# decissionTreeModel.fit(f_train, t_train)
# pred_t = decissionTreeModel.predict(f_test)
# print("Accuracy: ",m.accuracy_score(t_test,pred_t))
# print(decissionTreeModel.predict(f_test))

logRegModel = lr()
# logRegModel.fit(f_train, t_train)
# pred_t = logRegModel.predict(f_test)
# print(logRegModel.predict(f_test))
# print("Accuracy: ",m.accuracy_score(t_test,pred_t))

class StackingModel:
  def __init__(self, m1, m2, m3, *args):
    self.classifier = [m1, m2, m3]
    self.meta_model = gnb()
    self.train_x = args[0]
    self.test_x = args[1]
    self.train_y = args[2]
    self.test_y = args[3]
    self.pred = np.array([])
    self.new_test = np.array([])
    self.new_pred = np.array([])
  def train_together(self, pridict_accuracy = False):
    print("Stacking Ensemble Learnning")
    for i in self.classifier:
      i.fit(self.train_x, self.train_y)
    if pridict_accuracy:
      ind = 0
      for i in self.classifier:
        pred_t = i.predict(self.test_x)
        self.pred = np.concatenate([self.pred, pred_t])
        print("Accuracy: ",m.accuracy_score(self.test_y,pred_t))
        ind += 1
  def make_tts_on_pred_and_train(self, sepration = 0.5, random_shuffle = 0):
    print()
    self.new_test = np.concatenate([self.test_y, self.test_y, self.test_y])
    self.train_x, self.test_x, self.train_y, self.test_y = tts(self.pred,self.new_test)
    self.train_x = self.train_x.reshape(-1, 1)
    self.test_x = self.test_x.reshape(-1, 1)
    self.meta_model.fit(self.train_x, self.train_y)
    self.new_pred = np.concatenate([self.new_pred,self.meta_model.predict(self.test_x)])
    accu = m.accuracy_score(self.test_y,self.new_pred)
    print("New Pridicted: ", self.new_pred)
    print("Stacking Model Accuracy: ", accu)

sm = StackingModel(logRegModel, gaussionModel, decissionTreeModel, f_train, f_test, t_train, t_test)
sm.train_together(True)
sm.make_tts_on_pred_and_train()

Stacking Ensemble Learnning
Accuracy:  0.965034965034965
Accuracy:  0.916083916083916
Accuracy:  0.8951048951048951

New Pridicted:  [1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
Stacking Model Accuracy:  0.9444444444444444
